# Aperture Photometry

***

## Kernel Information and Read-Only Status

To run this notebook, please select the "Roman Calibration" kernel at the top right of your window.

This notebook is read-only. You can run cells and make edits, but you must save changes to a different location. We recommend saving the notebook within your home directory, or to a new folder within your home (e.g. <span style="font-variant:small-caps;">file > save notebook as > my-nbs/nb.ipynb</span>). Note that a directory must exist before you attempt to add a notebook to it.

## Imports
We are using standard libraries from the Numpy and Astropy stack.

- *astropy*
- *numpy*
- *copy*
- *matplotlib*
- *photutils* is an Astropy-affiliated package for photometry
- *roman_datamodels* opens and validates WFI data files
- *asdf* opens WFI data files
- *os* for checking if files exist
- *s3fs* streams data from Simple Storage Service (S3) buckets on Amazon Web Services (AWS)
- *stpsf* for retrieving the encircled energy profile of stars

In [ ]:
from astropy.table import Table
from astropy.stats import SigmaClip, sigma_clipped_stats
import asdf
import crds
import copy
import matplotlib.pyplot as plt
import numpy as np
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry, ApertureStats
import roman_datamodels as rdm
import os
import s3fs
import stpsf

## Introduction
This notebook explains how to perform **forced aperture photometry** (also known as *forced photometry* or *aperture photometry*) on Roman WFI images. Aperture photometry is useful for measuring the integrated fluxes for a set of specified source positions and aperture sizes. This technique is often valuable for extracting fluxes of known sources when it is impractical to fit for their positions and light distributions, or when simplicity and speed are high priorities. For example, aperture photometry can be used for:

- **Faint sources.** If a source is too faint, then it can be difficult to fit for the source centroid, light profile, and flux.
- **Sources detected at other wavelengths.** If a source is detected in a given bandpass, and the source size is expected to be the same across wavelengths, then it can be useful to define one aperture and measure the source fluxes across multiple filters.
- **Time-series evolution of sources.** If a source brightness is decreasing over time, then we can use earlier observations obtained when the source was bright to specify an aperture and extract the flux in a time series.

Here, we cover a simple example using the `photutils` package to extract fluxes from a catalog of known sources.

Please note that the WFI calibration is currently measured from ground test data, and changes are expected after launch. 

***

## Tutorial Data

### Image Data

In this tutorial, we use a Level 2 (L2; calibrated rate image) WFI data file that is the result of RomanCal processing of a Level 1 (L1; uncalibrated ramp cube) simulated file created with Roman I-Sim. If you have already worked through the tutorials "Simulating WFI Imaging Data with Roman I-Sim" and "Calibrating WFI Exposures with RomanCal," then you may already have these files saved locally. If not, then these files are also stored in the Nexus S3 bucket. For more information on how to access these data, see the [Data Discovery and Access](../data_discovery_and_access/data_discovery_and_access.ipynb) tutorial.

As a reminder, the file we are using is a L2 file meaning that the data were processed to flag and/or correct for detector-level effects (e.g., saturation, classic non-linearity, etc.), and that the ramp was fitted into a count rate image in units of Data Numbers (DN) per second.

In [ ]:
#Stream the files from the S3 bucket if they are not in local storage

if os.path.exists('r0003201001001001004_0001_wfi01_f106_cal.asdf'):
    f = rdm.open('r0003201001001001004_0001_wfi01_f106_cal.asdf')
    image = f.data.copy()
    wcs = copy.deepcopy(f.meta.wcs)
else:
    fs = s3fs.S3FileSystem()
    asdf_dir_uri = 's3://roman-sci-test-data-prod-summer-beta-test/AAS_WORKSHOP/'
    asdf_file_uri = asdf_dir_uri + 'r0003201001001001004_0001_wfi01_f106_cal.asdf'
    with fs.open(asdf_file_uri, 'rb') as f:
        af = asdf.open(f)
        dm = rdm.open(af).copy()
        image = dm.data.copy()
        err = dm.err.copy()
        dq = np.bool(dm.dq.copy())
        meta = copy.deepcopy(dm.meta)
        wcs = copy.deepcopy(dm.meta.wcs)

### Source Catalog

We also have access to a source catalog that was used to simulate the WFI image. It contains stars and galaxies, which are labeled as "PSF" and "SER" under the column `type`. Source fluxes are available in all WFI filters (F062, F087, F106, F129, F146, F158, F184, and F213) and are sampled from a lognormal distribution. Note that fluxes are all given in *maggies*, which are defined as ${\rm maggie} \equiv 10^{-0.4 m_{AB}}$, for an AB apparent magnitude $m_{AB}$. 

For galaxies, morphological parameters like `n` (Sersic index), `half_light_radius`, `pa` (position angle), and `ba` (axis ratio) are also provided in the catalog. These are sampled according to fiducial (and likely unrealistic) distributions.

In [ ]:
#Stream the files from the S3 bucket if they are not in local storage

if os.path.exists('full_catalog.ecsv'):
    cat = Table.read('full_catalog.ecsv')
else:
    fs = s3fs.S3FileSystem()
    asdf_dir_uri = 's3://roman-sci-test-data-prod-summer-beta-test/AAS_WORKSHOP/'
    asdf_file_uri = asdf_dir_uri + 'full_catalog.ecsv'
    with fs.open(asdf_file_uri, 'rb') as f:
        cat = Table.read(f, format='ascii.ecsv')

We can also display the first five rows and all columns of the catalog:

In [ ]:
cat[:5]

We can convert (RA, Dec) to (x, y) positions on the WFI01 detector.

In [ ]:
x_cat, y_cat = wcs.invert(cat["ra"].data, cat["dec"].data)
cat['x'] = x_cat
cat['y'] = y_cat

It might be helpful to quantify the number of sources there are before we try to visualize them! We first create a mask of sources that actually fall on the detector, and then divide the remaining sources into stars and galaxies.

In [ ]:
mask = (np.isfinite(x_cat) & np.isfinite(y_cat))
print(f"Number of sources on detector: {sum(mask)}")

stars = cat[mask & (cat["type"] == "PSF")]
galaxies = cat[mask & (cat["type"] == "SER")]
print(f"Number of stars: {len(stars)}\nNumber of galaxies: {len(galaxies)}")

We can now plot the distribution of source fluxes using a histogram. We will plot the flux from the input catalog in units of AB magnitudes. Recall that the input catalog is in units of maggies. The transformation from maggies to AB magnitudes is given by:

$m_{\mathrm{AB}} = -2.5 \times \log_{10}(f_{\mathrm{maggies}})$

In [ ]:
stars['abmag'] = -2.5 * np.log10(stars['F106'])
galaxies['abmag'] = -2.5 * np.log10(galaxies['F106'])

fig, ax = plt.subplots()
ax.hist(stars['abmag'], bins=50, range=[16, 24], log=True, label='Stars')
ax.hist(galaxies['abmag'], bins=50, range=[16, 24], log=True, label='Galaxies')

ax.set_xlabel(r'$m_{\mathrm{AB, F106}}$')
ax.set_ylabel('Number of Sources')
ax.xaxis.set_inverted(True)
ax.legend()

ax.grid(alpha=0.3)

## Forced Aperture Photometry

Forced aperture photometry uses predefined source positions to place apertures and measure the flux within them. We use the source catalog information provided above for this. If source positions are unavailable, you will need to perform source detection first to determine their locations (see [Additional Resources](#Additional-Resources)).

### Correcting for Pixel Area

First, we need to correct our data for geometric distortion, which causes pixels to have different areas projected on the sky. The WFI distortion is described by a 2-D polynomial, and the area of each pixel is given by the determinant of the Jacobian matrix of this polynomial. A reference file, called the Pixel Area Map (PAM), provides the area per pixel of each WFI detector normalized to the nominal pixel area. The nominal pixel area is the pixel area at the tangent point about which the distortion polynomial is defined.

First, we can use the `crds` Python package to request the best pixel area map reference file that matches our observation. In this case, CRDS needs to know the instrument name (WFI), detector name (WFI01), and the start time of the exposure. We can pull all of this information out of the metadata of the L2 file. Note that we need to specify the start time as a string, so we use `meta.exposure.start_time.isot` to convert the `astropy.time.Time` object in the metadata to an ISOT string (i.e., it has the format YYYY-MM-DDTHH:MM:SS.SSS). We also specify that the observatory is "roman" and that we want the "area" reference file type:

In [ ]:
crds_result = crds.getreferences({'ROMAN.META.INSTRUMENT.DETECTOR': meta.instrument.detector,
                             'ROMAN.META.INSTRUMENT.NAME': meta.instrument.name,
                             'ROMAN.META.EXPOSURE.START_TIME': meta.exposure.start_time.isot}, observatory='roman', reftypes=['area'])

print(crds_result)

We see that a dictionary is returned. The keys of the dictionary are the reference file types (e.g., "area"), and the values are the path to the correct file in your local CRDS cache. If the file is not in your cache, then the `crds.getreferences()` function will download the file to your local cache. At this time, CRDS does not support streaming reference files into memory from the CRDS server, but work is ongoing to adapt CRDS to cloud applications.

Next, let's take a look at the pixel area map that we now have:

In [ ]:
pam = rdm.open(crds_result['area'])

fig, ax = plt.subplots()
img = ax.imshow(pam.data, origin='lower')
ax.set_xlabel('X science coordinate (pixels)')
ax.set_ylabel('Y science coordinate (pixels)')
ax.set_title(f'Pixel Area Map for {pam.meta.instrument.detector}')
plt.colorbar(img);

Now let's multiply the science data and the uncertainty array by the PAM to adjust for the relative sizes of the pixels on the sky:

In [ ]:
image *= pam.data
err *= pam.data

### Create Apertures

For simplicity, we will focus on the stars from this point. From the catalogs, we now know the positions of every star. We can define set aperture radii in units of pixels; we choose radii of 3 pixels:

In [ ]:
star_positions = [(x, y) for y, x in zip(stars['y'].data, stars['x'].data)]
star_apertures = CircularAperture(positions=star_positions, r=3)

Let's create annulus apertures with $r_{in} = 20$ pixels and $r_{out} = 25$ pixels to measure local sky background values:

In [ ]:
star_bkg_apertures = CircularAnnulus(positions=star_positions, r_in=20, r_out=25)

### Source Visualization

Let's plot our apertures on a subsection of the image to visually confirm that we have placed our apertures correctly. There may be a few small, fuzzy galaxies in the image that are not marked since we are focusing on stars. Similarly, there may be several small features that are detector imperfections that are also not marked, but those will be masked in the data quality array.

In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))

# show the simulated image
ax.imshow(image, origin='lower', vmin=0, vmax=12, cmap='gray_r')

# plot circles over bright galaxies and stars
star_apertures.plot(color='cornflowerblue')
star_bkg_apertures.plot(color='cornflowerblue', ls=':')

# zoom in on 1/16th of the image
ax.set_xlim(1024, 1536)
ax.set_ylim(1024, 1536)

plt.axis('off')
plt.show()

### Aperture Photometry with Photutils

We can now perform aperture photometry on the selected sources. First we measure the photometry in a circular aperture on each source:

In [ ]:
columns = ('id', 'xcentroid', 'ycentroid', 'sum', 'sum_err')
star_phot = ApertureStats(image, star_apertures, error=err)
phot_tab = star_phot.to_table(columns)
phot_tab[:5]

And now let's do the same for the background annuli using the ApertureStats class. We will use the median of the pixels within the annulus as our estimate of the sky background **per pixel**. We will need to adjust this to the area of the star circular apertures (after accounting for the masked pixels).

In [ ]:
columns = ('id', 'xcentroid', 'ycentroid', 'median', 'std')
sigclip = SigmaClip(sigma=3.0, maxiters=5)
star_bkg = ApertureStats(image, star_bkg_apertures, sigma_clip=sigclip, mask=dq)
star_bkg.to_table(columns)[:5]

For our next steps, we will need the nominal pixel area in steradians (sr) and the zeropoint in megaJanskys per steradian (MJy/sr), both of which can be retrieved from the L2 metadata. Note that the pre-launch zeropoint is based on the unit response of the throughput curve of the integrated optical system (i.e., including all of the mirror reflectivities, optical element transmission, and detector quantum efficiency). The zeropoint also includes a factor of the median measured gain of the detector. The pre-launch zeropoint uncertainty is based solely on the standard deviation of the measured gain. After launch, these values will be updated using on-orbit observations of spectrophotometric standard stars and the photometric touchstone fields.

In [ ]:
zpt = meta.photometry.conversion_megajanskys
pixel_area = meta.photometry.pixel_area

print(f'Zeropoint = {zpt} (MJy/sr) / (DN/s)')
print(f'Nominal pixel area = {pixel_area} sr')

Next, we subtract the sky background values from the source fluxes and propagate the uncertainty in the background into the total uncertainty:

In [ ]:
flux_bkgsub = star_phot.sum - (star_bkg.median * star_apertures.area)
phot_tab['flux_jy'] = flux_bkgsub * zpt * 1e6 * pixel_area

We also need to correct our fluxes for the fraction of the starlight that falls outside of the aperture. The amount of light within the aperture as a function of the total light within a hypothetical infinite aperture is called the encircled energy and can be either measured from the image data using a series of apertures or can be determined from STPSF. In the future, a reference file in CRDS will also provide this information. For now, let's use STPSF:

In [ ]:
wfi = stpsf.WFI()
wfi.filter = meta.instrument.optical_element
wfi.detector = f'SCA{meta.instrument.detector[-2:]}'
wfi.detector_position = (2043.5, 2043.5)
wfi_psf = wfi.calc_psf()
ee_func = stpsf.measure_ee(wfi_psf)

With these settings, our encircled energy measurement is an average one for the whole detector, measured at the center of the detector, and for the default STPSF spectral energy distribution (SED), which is a 5700 K blackbody (similar to a G2V star). The variable `ee_func` above is a function that takes as its single argument the aperture radius in arcseconds and returns the percentage of the PSF contained within that aperture. The inverse of that encircled energy fraction is the correction factor that we must apply to our measurements. This is called the encircled energy correction or aperture correction. Let's calculate that now for a circular aperture with a 3-pixel radius and assuming a pixel scale of 0.11 arcseconds per pixel (i.e., radius = 0.33 arcseconds):

In [ ]:
ee_correction = 1/ee_func(3 * 0.11)
print(f'Encircled energy correction = {ee_correction}')

Now we can apply the aperture correction to our flux, and then convert the flux in Jy to maggies and AB magnitudes. For any source with a flux in maggies less than 0, we will assign a value of NaN for the AB magnitude.

In [ ]:
phot_tab['flux_jy'] *= ee_correction
phot_tab['flux_maggies'] = phot_tab['flux_jy'] / 3631

# Mask out things with flux <= 0
mask = phot_tab['flux_maggies'] <= 0
phot_tab['flux_maggies'][mask] = np.nan
phot_tab['flux_jy'][mask] = np.nan

# Convert the flux to AB magnitudes
phot_tab['abmag'] = -2.5 * np.log10(phot_tab['flux_maggies'])

phot_tab[:5]

Let's evaluate our results by plotting the difference between the simulated catalog and measured AB magnitudes. The blue points show the photometry of the stars, while the orange line shows the median offset between the input catalog and the measurements. The labeled uncertainty on the median is the standard deviation of the distribution. We also include a black line at an offset of 0 AB magnitudes as a reference.

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))

# Compute the offset between the input catalogs (stars) and measured (phot_tab) AB magnitudes and plot them
offset = stars['abmag'] - phot_tab['abmag']
ax.scatter(stars['abmag'], offset, s=5, color='cornflowerblue', alpha=0.3)

# Compute the median offset ignoring NaNs and plot the median and a reference line at 0
_, med, std = sigma_clipped_stats(offset[~np.isnan(offset)], sigma=5, maxiters=2)
ax.axhline(med, c='darkorange', label=fr'Median $\Delta m = {med:.3f}$ AB mag, Std. $\Delta m = {std:.3f}$ AB mag', zorder=1e4)
ax.axhline(0, color='black', zorder=-100)

# Restrict the axis limits for visibility. You may need to change these if you alter the input
# data for this tutorial.
ax.set_xlim(21, 16.5)
ax.set_ylim(-0.5, 0.5)

# Label the axes
ax.set_xlabel(r'$m_{\mathrm{AB, input}}$')
ax.set_ylabel(r'$m_{\mathrm{AB, input}} - m_{\mathrm{AB, measured}}$')
ax.legend(loc='lower right', fontsize=10)

# Overplot a reference grid
ax.grid(alpha=0.1)
plt.savefig('photometry.png')

Our measurements of stars in the image agree well with the input catalog fluxes at the level of approximately 0.9%. There could be several reasons why we see a small offset between the input and output magnitudes. First, we use a single value of the gain for our zeropoint, but we could do better and use a gain per pixel to convert the image from DN/s to photo-electrons per second and apply the zeropoint (with the gain removed). This would likely decrease the scatter in the plot. Secondly, we used a single aperture correction for the whole image based on the model of a star with an average SED and located at the center of the detector. A more detailed handling of the aperture correction may also decrease the scatter, and may also help bring the measurements into better alignment with the input catalog. Finally, a more careful treatment of other systematics (e.g., interpixel capacitance) may yield even better results.

A similar exercise in photometry can be repeated for the galaxies in the input catalog, but care should be taken when selecting apertures for galaxies as fixed aperture sizes will correspond to different physical sizes at the distances of galaxies. Other choices for galaxy apertures include, e.g., Kron apertures or isophotal apertures.

## Additional Resources
The [Photutils documentation](https://photutils.readthedocs.io/en/stable/) has additional tutorials for detecting and fitting sources in images:

- [Aperture photometry](https://photutils.readthedocs.io/en/stable/aperture.html)
- [Background estimation](https://photutils.readthedocs.io/en/stable/background.html)
- [Source detection](https://photutils.readthedocs.io/en/stable/detection.html)
- [PSF photometry](https://photutils.readthedocs.io/en/stable/psf.html)

## About This Notebook

**Author:** John F. Wu, Tyler Desjardins\
**Updated On:** 2025-05-28

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 